In [1]:
import os
import numpy as np
import pandas as pd
from keras.applications.densenet import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# Set the path to the dataset
data_dir = '/kaggle/input/animals-detection-images-dataset'

# Set the image size and batch size
image_size = (224, 224)
batch_size = 32

# Create an image data generator for the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Create a data generator for the training data
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

# Create an image data generator for the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# Create a data generator for the validation data
val_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

# Load the DenseNet121 model with pre-trained weights
base_model = DenseNet121(weights='imagenet', include_top=False)

# Add a global average pooling layer to the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a dense layer with the number of classes as the output size
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create a new model with the added layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model using the Adam optimizer and categorical crossentropy loss function
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up model checkpoint to save the best weights during training
checkpoint = ModelCheckpoint('animal_detection_model.h5', save_best_only=True)

# Train the model on the training data for 5 epochs, using the ModelCheckpoint callback
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[checkpoint])

Found 22566 images belonging to 80 classes.
Found 6505 images belonging to 80 classes.
29084464/29084464 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
706/706 [==============================] - 958s 1s/step - loss: 1.4365 - accuracy: 0.6396 - val_loss: 0.8591 - val_accuracy: 0.7500
Epoch 2/5
706/706 [==============================] - 791s 1s/step - loss: 0.7457 - accuracy: 0.7787 - val_loss: 0.7853 - val_accuracy: 0.7666
Epoch 3/5
706/706 [==============================] - 706s 1000ms/step - loss: 0.5828 - accuracy: 0.8121 - val_loss: 0.7488 - val_accuracy: 0.7742
Epoch 5/5
706/706 [==============================] - 690s 978ms/step - loss: 0.5455 - accuracy: 0.8200 - val_loss: 0.7735 - val_accuracy: 0.7783


In [2]:
from flask import Flask, request, jsonify
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

app = Flask(__name__)

# Load the trained model
model = load_model('animal_detection_model.h5')

# Define a route for making predictions
@app.route('/predict', methods=['POST'])
def predict():
    # Get the image from the request
    img = request.files['image']

    # Load the image and resize it to the required size
    img = image.load_img(img, target_size=(224, 224))

    # Convert the image to a numpy array
    img = image.img_to_array(img)

    # Expand the dimensions of the image to match the expected input shape of the model
    img = np.expand_dims(img, axis=0)

    # Make a prediction using the model
    prediction = model.predict(img)

    # Get the index of the predicted class
    predicted_class = np.argmax(prediction[0])

    # Get the name of the predicted class
    class_names = ['cat', 'dog', 'horse']
    predicted_class_name = class_names[predicted_class]

    # Return the prediction as a JSON response
    return jsonify({'prediction': predicted_class_name})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off
